In [1]:
from __future__ import unicode_literals
from hazm import *
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense
import numpy as np

import re
from urlextract import URLExtract
import emojis

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
import np_utils

from tqdm import tqdm

In [2]:
data = pd.read_csv('data/Digikala.csv', on_bad_lines='skip')

In [3]:
data.head()

,Text,Score,Suggestion
0,این اولین تجربه من برای خرید ایفون هست امروز...,100,1
1,خرید این محصول رو توصیه میکنم,84,1
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,60,1
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,96,1
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,92,1


In [4]:
data['Suggestion'].value_counts()

1    2382
3     460
2     419
Name: Suggestion, dtype: int64

In [5]:
def replace_values(value):
    if value == 2 or value == 3:
        return 0
    return value

data['Suggestion'] = data['Suggestion'].apply(replace_values)

In [6]:
data['Suggestion'].value_counts()

1    2382
0     879
Name: Suggestion, dtype: int64

In [7]:
def switch_values(value):
    if value == 0:
        return 1
    elif value == 1:
        return 0
    return value

data['Suggestion'] = data['Suggestion'].apply(switch_values)

In [8]:
data['Suggestion'].value_counts()

0    2382
1     879
Name: Suggestion, dtype: int64

In [9]:
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_fa_numbers(input_str): # arabic numeral to global conversion
    mapping = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9',
        '.': '.',
    }
    return _multiple_replace(mapping, input_str)


def convert_ar_characters(input_str): # identical char conversion
    
    mapping = {
        'ك': 'ک',
        'ى': 'ی',
        'ي': 'ی',
        'ئ':'ی',
        'إ':'ا',
        'أ':'ا',
        'ة':'ه',
        'ؤ':'و'
    }
    return _multiple_replace(mapping, input_str)


def preprocess(text):
    extractor = URLExtract()
    for url in extractor.gen_urls(text):
        text = text.replace(url,'<URL>') # omitting the urls and replacing them with a cons
    emj = emojis.get(text)
    for i in emj:
        if i in text:
            text = text.replace(i,'<emoji>') # same as the url for the emojis
    text = convert_fa_numbers(text)
    text = convert_ar_characters(text)
    # regex to detect and replace all smilies in the text with <smiley>
    text = re.sub(r"(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|:\s?D|8-\)|:\s?\||;\s?\)|:-\*|:-\||:-\(|:\s?P|:-P|:-p|:-b|:-O|:-o|:-0|:-\@|:\$|:-\^|:-&|:-\*|:-\+|:-\~|:-\`|:-\>|:-\<|:-\}|:-\{|\[:\s?\]|\[:\s?\]|:\s?\]|:\s?\[|:\s?\}|:\s?\{)",'<smiley>',text)
    text = text.lower()
    text = text.strip()
    text = re.sub(r'[<>#.:()"\'!?؟،,@$%^&*_+\[\]/]', ' ', text)
    text = re.sub(r'[\s]{2,}', ' ', text)
    text = re.sub(r'(\w)\1{2,}', r'\1',text)
    if re.search(r'[\u0600-\u06FF]', text):
        return(text)
    else:
        return 'None'

In [10]:
tqdm.pandas()

In [11]:
data['cleaned'] = data['Text'].progress_apply(preprocess)

100%|██████████████████████████████████████████████████████████████████████████████| 3261/3261 [00:43<00:00, 75.53it/s]


In [12]:
 data.head()

,Text,Score,Suggestion,cleaned
0,این اولین تجربه من برای خرید ایفون هست امروز...,100,0,این اولین تجربه من برای خرید ایفون هست امروز ب...
1,خرید این محصول رو توصیه میکنم,84,0,خرید این محصول رو توصیه میکنم
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,60,0,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ندیدم
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,96,0,سلام خدمت دوستان این گوشی از همه نظر عالی کیفی...
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,92,0,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...


In [13]:
data = data.dropna()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3261 entries, 0 to 3260
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Text        3261 non-null   object
 1   Score       3261 non-null   int64 
 2   Suggestion  3261 non-null   int64 
 3   cleaned     3261 non-null   object
dtypes: int64(2), object(2)
memory usage: 102.0+ KB


In [15]:
data.to_csv('data/digikala_preprocessed.csv')

In [100]:
data = pd.read_csv('data/digikala_preprocessed.csv', on_bad_lines='skip')

In [101]:
data['Score'].value_counts()

60     503
100    492
88     266
84     260
92     236
80     225
96     199
76     167
72     161
68     158
64      86
52      78
56      77
48      66
44      42
40      41
0       37
36      36
28      24
20      23
32      18
24      17
16      13
4       10
8        8
12       8
94       5
90       2
82       1
10       1
98       1
Name: Score, dtype: int64

In [102]:
def replace_values_with_status(df, column_name):
    df[column_name] = df[column_name].apply(lambda value: 'UNSAT' if value <= 40 else 'SAT')
    return df

data = replace_values_with_status(data, 'Score')

In [105]:
data['Score']

0       SAT
1       SAT
2       SAT
3       SAT
4       SAT
       ... 
3256    SAT
3257    SAT
3258    SAT
3259    SAT
3260    SAT
Name: Score, Length: 3261, dtype: object

In [106]:
data['Score'].value_counts()

SAT      3025
UNSAT     236
Name: Score, dtype: int64

In [107]:
data.head()

,Unnamed: 0,Text,Score,Suggestion,cleaned
0,0,این اولین تجربه من برای خرید ایفون هست امروز...,SAT,0,این اولین تجربه من برای خرید ایفون هست امروز ب...
1,1,خرید این محصول رو توصیه میکنم,SAT,0,خرید این محصول رو توصیه میکنم
2,2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,SAT,0,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ندیدم
3,3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,SAT,0,سلام خدمت دوستان این گوشی از همه نظر عالی کیفی...
4,4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,SAT,0,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...


In [108]:
def set_suggestion(row):
    if row['Score'] == 'UNSAT':
        return 1
    else:
        return 0
    
data['Suggestion'] = data.apply(set_suggestion, axis=1)

In [109]:
data = data.drop('Unnamed: 0', axis=1)

In [110]:
data.head()

,Text,Score,Suggestion,cleaned
0,این اولین تجربه من برای خرید ایفون هست امروز...,SAT,0,این اولین تجربه من برای خرید ایفون هست امروز ب...
1,خرید این محصول رو توصیه میکنم,SAT,0,خرید این محصول رو توصیه میکنم
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,SAT,0,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ندیدم
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,SAT,0,سلام خدمت دوستان این گوشی از همه نظر عالی کیفی...
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,SAT,0,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...


In [111]:
count_vectorizer = CountVectorizer()
X_count_vectorized = count_vectorizer.fit_transform(data.cleaned).todense()

In [112]:
vectorizer = TfidfVectorizer(min_df=2, max_features= 10000)
X_tfidf_vectorized = vectorizer.fit_transform(data.cleaned).todense()

In [113]:
labels = data['Suggestion'].values

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_count_vectorized, labels, test_size=0.2, random_state=42) 

In [115]:
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(X_tfidf_vectorized, labels, test_size=0.2, random_state=42)

In [116]:
input_dim = X_tfidf_train.shape

In [117]:
print(input_dim)

(2608, 5872)


In [118]:
classifier = LogisticRegression()
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.9356814701378254


In [119]:
clf = LogisticRegression()
X_tfidf_train = np.asarray(X_tfidf_train)
y_tfidf_train = np.asarray(y_tfidf_train)
X_tfidf_test = np.asarray(X_tfidf_test)
y_tfidf_test = np.asarray(y_tfidf_test)
clf.fit(X_tfidf_train, y_tfidf_train)
tfidf_score = clf.score(X_tfidf_test, y_tfidf_test)
print("Accuracy:", tfidf_score)

Accuracy: 0.9402756508422665


In [120]:
nb_classes = 2
batch_size = 32
nb_epochs = 10

In [121]:
from tensorflow.keras.utils import to_categorical

In [122]:
y_tfidf_train_cat = to_categorical(y_tfidf_train,)

In [123]:
model = Sequential()

model.add(Dense(1000,input_shape= (input_dim[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(500))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(50))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [124]:
tf.config.run_functions_eagerly(True)

In [125]:
model.fit(X_tfidf_train, y_tfidf_train_cat, batch_size=batch_size, epochs=nb_epochs,verbose=2)

Epoch 1/10


C:\Users\Hexa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


82/82 - 8s - loss: 0.3209 - 8s/epoch - 97ms/step
Epoch 2/10
82/82 - 8s - loss: 0.1936 - 8s/epoch - 95ms/step
Epoch 3/10
82/82 - 8s - loss: 0.1316 - 8s/epoch - 94ms/step
Epoch 4/10
82/82 - 8s - loss: 0.0666 - 8s/epoch - 95ms/step
Epoch 5/10
82/82 - 8s - loss: 0.0228 - 8s/epoch - 95ms/step
Epoch 6/10
82/82 - 8s - loss: 0.0059 - 8s/epoch - 93ms/step
Epoch 7/10
82/82 - 8s - loss: 0.0040 - 8s/epoch - 95ms/step
Epoch 8/10
82/82 - 8s - loss: 0.0032 - 8s/epoch - 95ms/step
Epoch 9/10
82/82 - 8s - loss: 0.0015 - 8s/epoch - 94ms/step
Epoch 10/10
82/82 - 8s - loss: 0.0022 - 8s/epoch - 96ms/step


In [126]:
y_test_pred = model.predict(X_tfidf_test)
y_test_predclass = np.argmax(y_test_pred, axis=1)
y_trian_pred = model.predict(X_tfidf_train)
y_train_predclass = np.argmax(y_trian_pred, axis=1)

82/82 [==============================] - 1s 7ms/step


In [127]:
from sklearn.metrics import accuracy_score,classification_report
print ("nDeep Neural Network - Test accuracy:",(round(accuracy_score(y_tfidf_test, y_test_predclass),4)*100))
print ("nDeep Neural Network - Train accuracy:",(round(accuracy_score(y_tfidf_train, y_train_predclass),4)*100))

nDeep Neural Network - Test accuracy: 92.96
nDeep Neural Network - Train accuracy: 99.96000000000001


In [128]:
from tensorflow.keras.models import save_model

model.save('model/digikala_keras_model.h5')

C:\Users\Hexa\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [139]:
X_pred = vectorizer.transform([preprocess('اصلا پیشنهاد نمیشود')]).todense()

In [140]:
model.predict(X_pred)

1/1 [==============================] - 0s 16ms/step


array([[0.47478804, 0.525212  ]], dtype=float32)